<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


---




<p align="center"><h1 align="center">Quick Start: Flower Image Classification Tutorial</h1> 

---

<h3 align="center">(Prepare to deploy model and preprocessor to an AI Model Share Model Playground REST API<br> and Web Dashboard in four easy steps...)</h3></p>
<p align="center"><img width="100%" src="https://aimodelsharecontent.s3.amazonaws.com/ModelandPreprocessorObjectPreparation.jpeg" /></p>


---



## **Credential Configuration**

In order to deploy an AI Model Share Model Playground, you will need a credentials text file. 

Generating your credentials file requires two sets of information: 
1. Your AI Model Share username and password (create them [HERE](https://www.modelshare.org/login)). 
2. Your AWS (Amazon Web Services) access keys (follow the tutorial [HERE](https://docs.aws.amazon.com/general/latest/gr/aws-sec-cred-types.html)). 

You only need to generate your credentials file once. After running the configure function below, save the outputted file for all your future Model Playground deployments and competition submissions. 

*Note: Handle your credentials file with the same level of security you handle your passwords. Do not share your file with anyone, send via email, or upload to Github.*


In [ ]:
#install aimodelshare library
! pip install --extra-index-url https://test.pypi.org/simple/ --upgrade aimodelshare


In [ ]:
# Generate credentials file 
import aimodelshare as ai 
from aimodelshare.aws import configure_credentials 

configure_credentials()

AI Modelshare Username:··········
AI Modelshare Password:··········
AWS_ACCESS_KEY_ID:··········
AWS_SECRET_ACCESS_KEY:··········
AWS_REGION:··········
Configuration successful. New credentials file saved as 'credentials.txt'


## **Set up Environment**

Use your credentials file to set your credentials for all aimodelshare functions. 

In [2]:
# Set credentials 
import aimodelshare as ai
from aimodelshare.aws import set_credentials

set_credentials(credential_file="credentials.txt", type="deploy_model")

AI Model Share login credentials set successfully.
AWS credentials set successfully.


In [3]:
# Get materials for tutorial
import aimodelshare as ai
keras_model, model_2, target_labels, X_test, y_test, y_test_labels = ai.import_quickstart_data()

: Downloading [================================================>]

Data downloaded successfully.

Preparing downloaded files for use...

Success! Your Quick Start materials have been downloaded. 
You are now ready to run the tutorial.


## **(1) Preprocessor Function & Setup**

### **Write a Preprocessor Function**


> ###   Preprocessor functions are used to preprocess data into the precise data your model requires to generate predictions.  

*  *Preprocessor functions should always be named "preprocessor".*
*  *You can use any Python library in a preprocessor function, but all libraries should be imported inside your preprocessor function.*  
*  *For image prediction models users should minimally include function inputs for an image filepath and values to reshape the image height and width.*  



In [4]:
# Here is a pre-designed preprocessor, but you could also build your own to prepare the data differently

def preprocessor(image_filepath, shape=(192, 192)):
        """
        This function preprocesses reads in images, resizes them to a fixed shape and
        min/max transforms them before converting feature values to float32 numeric values
        required by onnx files.
        
        params:
            image_filepath
                full filepath of a particular image
                      
        returns:
            X
                numpy array of preprocessed image data
                  
        """
           
        import cv2
        import numpy as np

        "Resize a color image and min/max transform the image"
        img = cv2.imread(image_filepath) # Read in image from filepath.
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 reads in images in order of blue green and red, we reverse the order for ML.
        img = cv2.resize(img, shape) # Change height and width of image.
        img = img / 255.0 # Min-max transform.


        # Resize all the images...
        X = np.array(img)
        X = np.expand_dims(X, axis=0) # Expand dims to add "1" to object shape [1, h, w, channels] for keras model.
        X = np.array(X, dtype=np.float32) # Final shape for onnx runtime.
        return X

## **(2) Train Model Using tf.keras (or Your Preferred ML Library)**

### Keras **Convolutional Neural Network**

In [5]:
#Here is a pre-trained keras_model, but you could also train your own model after preprocessing data with your preprocessor function.
keras_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 96, 96, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 96, 96, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 294912)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               29491300  
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                

## **(3) Save Preprocessor**
### Saves preprocessor function to "preprocessor.zip" file

In [6]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

In [7]:
#  Now let's import and test the preprocessor function to see if it is working...

import aimodelshare as ai
prep=ai.import_preprocessor("preprocessor.zip")

prep("quickstart_materials/example_data/100080576_f52e8ee070_n.jpg").shape

(1, 192, 192, 3)

## **(4) Save Keras Model to Onnx File Format**


In [8]:
# Save tf.keras model (or any tensorflow model) to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(keras_model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [9]:
#Set up arguments for Model Playground deployment
import pandas as pd 

model_filepath="model.onnx"
preprocessor_filepath="preprocessor.zip"
exampledata_filepath = "quickstart_materials/example_data" 

In [10]:
from aimodelshare import ModelPlayground

#Instantiate ModelPlayground() Class

myplayground=ModelPlayground(model_type="image", classification="TRUE", private="FALSE")

# Create Model Playground (generates live rest api and web-app for your model/preprocessor)

myplayground.deploy(model_filepath, preprocessor_filepath, target_labels, exampledata_filepath) 

We need some information about your model before we can build your REST API and interactive Model Playground.
   
Model Name (for AI Model Share Website):Flower Classification Competition
Model Description (Explain what your model does and 
 why end-users would find your model useful):This model classifies flower images into one of five flower labels.
Model Key Words (Search categories that describe your model, separated with commas):flower, classification, prediction
   
Creating your prediction API. (This process may take several minutes.)

[=====================================] Progress: 100% - Complete!                                            

Success! Your Model Playground was created in 71 seconds. 
 Playground Url: "https://d3i27i7803.execute-api.us-east-1.amazonaws.com/prod/m"

You can now use your Model Playground.

Follow this link to explore your Model Playground's functionality
You can make predictions with the Dashboard and access example code from the Programmatic ta

## **Use your new Model Playground!**

Follow the link in the output above to:
- Generate predictions with your interactive web dashboard
- Access example code in Python, R, and Curl

Or, follow the rest of the tutorial to create a competition for your Model Playground and: 
- Access verified model performance metrics 
- Upload multiple models to a leaderboard 
- Easily compare model performance & structure 

## **Part 2: Create a Competition**

-------

After deploying your Model Playground, you can now create a competition. 

Creating a competition allows you to:
1. Verify the model performance metrics on aimodelshare.org
2. Submit models to a leaderboard
3. Grant access to other users to submit models to the leaderboard
4. Easily compare model performance and structure 

In [11]:
# Create Competition
myplayground.create_competition(data_directory='data-directory-example', 
                                y_test = y_test_labels, 
                                generate_credentials_file = True) 


Enter competition name:Flower Competition
Enter competition description:Submit new models to improve prediction of flower images.
Enter data description (i.e.- filenames denoting training and test data, file types, and any subfolders where files are stored):Master folder with flower images, subfolders labeled by flower type.
Uploading your data. Please wait for a confirmation message.

 Success! Model competition created. 

Your team members can now make use of the following functions: 
submit_model() to submit new models to the competition leaderboard. 
download_data('public.ecr.aws/y2e2a1d6/data-directory-example-repository:latest') to download your competition data.  

You may update your prediction API runtime model with the update_runtime_model() function.

To upload new models and/or preprocessors to this API, team members should use 
the following credentials:

#Credentials for Competition: d3i27i7803
[submit_model:"https://d3i27i7803.execute-api.us-east-1.amazonaws.com/prod/m"]

In [12]:
#Instantiate Competition 
mycompetition= ai.Competition(myplayground.playground_url)

Submit Models

In [13]:
#Submit Model 1: 

# generate predicted y values (Model 1)
y_pred = keras_model.predict(X_test).argmax(axis=1)
prediction_labels = [y_test.columns[i] for i in y_pred]

#submit
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

{'accuracy': 0.5858310626702997, 'f1_score': 0.5805722863061182, 'precision': 0.6029215051441896, 'recall': 0.5875148809523809, 'mse': None, 'rmse': None, 'mae': None, 'r2': None}
Insert search tags to help users find your model (optional): conv2d layers, dropout, fc layers, flower, classification
Provide any useful notes about your model (optional): conv2d layers, dropout, fc layers
Share code: Insert AI Model Share url to Jupyter notebook with model code(optional): 
Share code: Insert Github url to Jupyter notebook with model code(optional): 


'Your model has been submitted as model version 1'

In [16]:
# Have a look at architecture for model two
model_2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 192, 192, 32)      416       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 192, 192, 32)      4128      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 96, 96, 16)        2064      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 96, 96, 16)        1040      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 48, 48, 16)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 48, 48, 16)       

In [14]:
# Save Model 2 to .onnx file

# Save tf.keras model to ONNX file
onnx_cnn2 = model_to_onnx(model_2, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

# Save model to local .onnx file
with open("model_2.onnx", "wb") as f:
    f.write(onnx_cnn2.SerializeToString()) 

In [17]:
# Submit Model 2

# generate predicted y values (Model 2)
y_pred = model_2.predict(X_test).argmax(axis=1)
prediction_labels = [y_test.columns[i] for i in y_pred]

#submit
mycompetition.submit_model(model_filepath = "model_2.onnx",
                                 prediction_submission=prediction_labels,
                                 preprocessor_filepath="preprocessor.zip")

{'accuracy': 0.5299727520435967, 'f1_score': 0.5135818973402516, 'precision': 0.5646184901731715, 'recall': 0.513263888888889, 'mse': None, 'rmse': None, 'mae': None, 'r2': None}
Insert search tags to help users find your model (optional): multiple stacked conv2d layers, dropout, FC layers,flower, classification
Provide any useful notes about your model (optional): adding more conv2d layers, stacking, dropout, FC
Share code: Insert AI Model Share url to Jupyter notebook with model code(optional): 
Share code: Insert Github url to Jupyter notebook with model code(optional): 


'Your model has been submitted as model version 2'

Get Leaderboard

In [18]:
data = mycompetition.get_leaderboard()
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,conv2d_layers,dense_layers,dropout_layers,flatten_layers,maxpooling2d_layers,relu_act,softmax_act,loss,optimizer,model_config,memory_size,username,version
0,58.58%,58.06%,60.29%,58.75%,keras,False,True,Sequential,7,29492221,1,2,2,1,1,2,1,function,RMSprop,"{'name': 'sequential_1', 'laye...",211112,mikedparrott,1
1,53.00%,51.36%,56.46%,51.33%,keras,False,True,Sequential,11,1851153,4,2,2,1,2,5,1,function,RMSprop,"{'name': 'sequential_2', 'laye...",303872,mikedparrott,2


Compare Models

In [19]:
# Compare two or more models (Experimental, Git-like Diffs for Model Architectures)
mycompetition.compare_models([1,2])

,Model_1_Layer,Model_1_Shape,Model_1_Params,Model_2_Layer,Model_2_Shape,Model_2_Params
0,Conv2D,"(None, 192, 192, 32)",416.000000,nan,nan,nan
1,MaxPooling2D,"(None, 96, 96, 32)",0.000000,nan,nan,nan
2,Dropout,"(None, 96, 96, 32)",0.000000,nan,nan,nan
3,Flatten,"(None, 294912)",0.000000,nan,nan,nan
4,Dense,"(None, 100)",29491300.000000,nan,nan,nan
5,Dropout,"(None, 100)",0.000000,nan,nan,nan
6,Dense,"(None, 5)",505.000000,nan,nan,nan
0,nan,nan,nan,Conv2D,"[None, 192, 192, 32]",416.000000
1,nan,nan,nan,Conv2D,"[None, 192, 192, 32]",4128.000000
10,nan,nan,nan,Dense,"[None, 5]",255.000000


Instantiate Model

In [21]:
mymodel = mycompetition.instantiate_model(2, trained=True)
print(mymodel.summary())
mymodel.predict(X_test)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 192, 192, 32)      416       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 192, 192, 32)      4128      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 96, 96, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 96, 96, 16)        2064      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 96, 96, 16)        1040      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 48, 48, 16)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 48, 48, 16)       

array([[0.17464854, 0.08563613, 0.32331744, 0.03516423, 0.3812337 ],
       [0.03796328, 0.15909852, 0.04345102, 0.64297384, 0.1165133 ],
       [0.07401693, 0.0260284 , 0.27283636, 0.02794363, 0.59917474],
       ...,
       [0.13816705, 0.0473027 , 0.2506315 , 0.03810798, 0.52579075],
       [0.07116339, 0.0122371 , 0.40838602, 0.0050164 , 0.5031971 ],
       [0.2223364 , 0.43105513, 0.06759172, 0.15008667, 0.12893012]],
      dtype=float32)

Inspect Y Test

In [22]:
mycompetition.inspect_y_test()

{'class_balance': {'daisy': 126,
  'dandelion': 180,
  'roses': 128,
  'sunflowers': 140,
  'tulips': 160},
 'class_labels': ['dandelion', 'roses', 'daisy', 'tulips', 'sunflowers'],
 'label_dtypes': {"<class 'str'>": 734},
 'y_length': 734,
 'ytest_example': ['daisy', 'sunflowers', 'roses', 'dandelion', 'roses']}